# Fashion Image Classification

Trabalho final da disciplina de Engenharia de Sistemas de Software Inteligentes do curso de Engenharia de Software da Pontifícia Universidade Católica do Rio de Janeiro (PUC-Rio).

## Descrição do Projeto

Este projeto é modelo de Machine Learning para classificação de imagens de moda. O objetivo é permitir que os usuários enviem imagens e recebam de volta a classificação correspondente.

### Etapa 1: Importação das bibliotecas necessárias

In [ ]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import pickle

from PIL import Image
import io

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### Etapa 2: Carga do Dataset

#### Dataset de Treinamento

In [ ]:
train_url = "./datasets/train.parquet"

train_dataset = pd.read_parquet(train_url)

# dataset muito grande, vamos usar apenas uma amostra de 10.000 linhas
train_dataset = train_dataset.sample(n=8000, random_state=42)

# Exibe as primeiras linhas do dataset
train_dataset.head()

#### Dataset de Testes

In [ ]:
test_url = "./datasets/test.parquet"

# já o dataset de teste é menor, vamos usar todas as linhas
test_dataset = pd.read_parquet(test_url)

# Exibe as primeiras linhas do dataset
test_dataset.head()

### Etapa 3: Separação em Conjuntos (Holdout)

In [ ]:
test_size = 0.20 # tamanho do conjunto de teste
seed = 7 # semente aleatória

# Extrai os bytes das imagens e converte para arrays numpy
def bytes_to_array(img_bytes):
  img = Image.open(io.BytesIO(img_bytes))
  img = img.resize((64, 64))  # Redimensiona para 64x64 pixels
  return np.array(img).flatten()  # Achata para vetor 1D

# Separação dos dados em treino e teste
X_train = np.stack(train_dataset['image'].apply(lambda x: bytes_to_array(x['bytes'])))
y_train = train_dataset['label'].values

X_test = np.stack(test_dataset['image'].apply(lambda x: bytes_to_array(x['bytes'])))
y_test = test_dataset['label'].values

# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 3

# Ajusta o número de partições para não exceder o número mínimo de amostras por classe
min_samples_per_class = np.min(np.bincount(y_train))
num_particoes = min(num_particoes, min_samples_per_class)
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # validação cruzada com estratificação

### Etapa 4: Criação e avaliação de modelos: linha base

In [ ]:
np.random.seed(7) # definindo uma semente global

# Lista que armazenará os modelos
models = []

# Criando os modelos e adicionando-os na lista de modelos
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Listas para armazenar os resultados
results = []
names = []

# Avaliação dos modelos
for name, model in models:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(15,10))
fig.suptitle('Comparação dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

### Etapa 5: Criação e avaliação de modelos: dados padronizados e normalizados

In [ ]:
np.random.seed(7) # definindo uma semente global para este bloco

# Listas para armazenar os armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []

# Criando os elementos do pipeline

# Algoritmos que serão utilizados
#knn = ('KNN', KNeighborsClassifier())
#cart = ('CART', DecisionTreeClassifier())
#naive_bayes = ('NB', GaussianNB())
svm = ('SVM', SVC())

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

# Montando os pipelines

# Dataset original
#pipelines.append(('KNN-orig', Pipeline([knn])))
#pipelines.append(('CART-orig', Pipeline([cart])))
#pipelines.append(('NB-orig', Pipeline([naive_bayes])))
pipelines.append(('SVM-orig', Pipeline([svm])))

# Dataset Padronizado
#pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
#pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
#pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))

# Dataset Normalizado
#pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
#pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
#pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))

# Executando os pipelines
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 3 casas decimais
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(25,6))
fig.suptitle('Comparação dos Modelos - Dataset orginal, padronizado e normalizado')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names, rotation=90)
plt.show()

### Etapa 6: Otimização dos hiperparâmetros

In [7]:
# Otimização dos hiperparâmetros para SVM padronizado usando GridSearchCV

# Pipeline com padronização e SVM
pipeline_svm = Pipeline([
  ('StandardScaler', standard_scaler[1]),
  ('SVM', SVC())
])

# Espaço de busca dos hiperparâmetros
param_grid = {
  'SVM__C': [0.1, 1, 10],
  'SVM__gamma': [0.001, 0.01, 0.1, 1],
  'SVM__kernel': ['rbf', 'linear']
}

# GridSearchCV
grid = GridSearchCV(estimator=pipeline_svm, param_grid=param_grid, cv=kfold, scoring=scoring, n_jobs=-1, verbose=2)
grid_result = grid.fit(X_train, y_train)

# Exibe os melhores parâmetros e a melhor acurácia
print(f"Melhor acurácia: {grid_result.best_score_:.4f}")
print("Melhores parâmetros:", grid_result.best_params_)

[CV] END ...SVM__C=0.1, SVM__gamma=0.001, SVM__kernel=linear; total time= 1.2min
[CV] END ...SVM__C=0.1, SVM__gamma=0.001, SVM__kernel=linear; total time= 1.2min
[CV] END ...SVM__C=0.1, SVM__gamma=0.001, SVM__kernel=linear; total time= 1.2min
[CV] END ....SVM__C=0.1, SVM__gamma=0.01, SVM__kernel=linear; total time= 1.1min
[CV] END ....SVM__C=0.1, SVM__gamma=0.01, SVM__kernel=linear; total time= 1.1min
[CV] END ....SVM__C=0.1, SVM__gamma=0.01, SVM__kernel=linear; total time= 1.1min
[CV] END ......SVM__C=0.1, SVM__gamma=0.001, SVM__kernel=rbf; total time= 4.4min
[CV] END ......SVM__C=0.1, SVM__gamma=0.001, SVM__kernel=rbf; total time= 4.4min
[CV] END ......SVM__C=0.1, SVM__gamma=0.001, SVM__kernel=rbf; total time= 4.4min
[CV] END .....SVM__C=0.1, SVM__gamma=0.1, SVM__kernel=linear; total time= 1.0min
[CV] END .....SVM__C=0.1, SVM__gamma=0.1, SVM__kernel=linear; total time=  59.9s
[CV] END .......SVM__C=0.1, SVM__gamma=0.01, SVM__kernel=rbf; total time= 6.2min
[CV] END .......SVM__C=0.1, 

### Etapa 7: Finalização do Modelo

In [8]:
# Treinando o modelo SVM com os melhores hiperparâmetros encontrados pelo GridSearchCV
best_svm = grid_result.best_estimator_
best_svm.fit(X_train, y_train)

# Avaliando no conjunto de teste
y_pred = best_svm.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")

Acurácia no conjunto de teste: 0.8293


In [9]:
# Treinando o modelo SVM otimizado com todo o dataset de treino disponível (X, y)
final_model = grid_result.best_estimator_
final_model.fit(X_train, y_train)
print("Modelo final treinado com todo o dataset de treino disponível.")

Modelo final treinado com todo o dataset de treino disponível.


### Etapa 8: Simulando a aplicação do modelo em dados não vistos

In [ ]:
# Teste do modelo com o dataset de teste

# Carrega imagens de teste
test_images = [
    './imgs/original/dress.jpg',
    './imgs/original/pullover.jpg',
    './imgs/original/sandal.jpg',
]

# Função para converter imagens de teste em arrays numpy
def load_test_images(image_paths):
    images = []
    for path in image_paths:
        img = Image.open(path).convert('L')  # Converte para escala de cinza
        img = img.resize((64, 64))  # Redimensiona para 64x64 pixels
        images.append(np.array(img).flatten())  # Achata para vetor 1D
    return np.array(images)

# Carrega as imagens de teste
test_images_array = load_test_images(test_images)

predictions = final_model.predict(test_images_array)

descriptions = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

# Mapeia as previsões para as descrições
predictions = [descriptions[pred] for pred in predictions]

# Exibe as previsões
for i in range(len(predictions)):
    print(f"Imagem {i+1}: {predictions[i]}")

Imagem 1: T-shirt/top
Imagem 2: Bag
Imagem 3: Sandal


### Etapa 9: Exportando o modelo para o formato PKL

In [ ]:
# Nome do arquivo para salvar o modelo
filename = 'fashion_image_classification_model.pkl'

# Salvar o modelo no arquivo
# pickle.dump(final_model, open(filename, 'wb'))

print(f"Modelo exportado para '{filename}'")